In [1]:
import pandas as pd
#!pip install psycopg2-binary
import psycopg2
from psycopg2 import Error
import numpy as np
import pickle
from os import path
#!pip install paramiko
#!pip install scp
from paramiko import SSHClient,AutoAddPolicy
from scp import SCPClient
from tqdm import tqdm

In [2]:
# Load raw dataset data
# Better to load in the order proposed by the lines and the comments
# Remember to merge common info between the datasets after importing an entire family of datasets

#dataset_name = 'UJI1'
#dataset_name = 'DSI1' # DSI1 DSI2 'DSI_trajectories'
#dataset_name = 'LIB1' # LIB1 LIB2
#dataset_name = 'SIM001' # SIM001 # SIM002 # SIM003 # SIM004 # SIM005 # SIM006 # SIM007 # SIM008 # SIM009 # SIM010 
#dataset_name = 'UTS'
#dataset_name = 'TUT3' # TUT3 TUT4 TUT1 TUT6 (then merge info between datasets)
#dataset_name = 'TUT2' # TUT2 TUT5 TUT7 (then merge info between datasets)
#dataset_name = 'IPIN21_Track3'




# Mandatory files

dataset_metadata = pd.read_csv('../Datasets/converted_datasets/' + dataset_name + '/dataset_metadata.csv', sep=',', header=None, na_values = 'NULL', keep_default_na=False)
dataset_url = dataset_metadata.iloc[0,0]
dataset_notes = dataset_metadata.iloc[0,1]


fingerprints_exists = False
places_exists = False

if path.exists('../Datasets/converted_datasets/' + dataset_name + '/fingerprints.csv'):
    fingerprints_frame = pd.read_csv('../Datasets/converted_datasets/' + dataset_name + '/fingerprints.csv', na_values = 'NULL', sep=',', dtype=object)
    fingerprints_exists = True
    
if path.exists('../Datasets/converted_datasets/' + dataset_name + '/places.csv'):
    places_frame = pd.read_csv('../Datasets/converted_datasets/' + dataset_name + '/places.csv', na_values = 'NULL', sep=',', dtype=object)
    places_exists = True
    




# Optional files
devices_exists = False
users_exists = False
tessellations_exists = False
wifiobs_exists = False
bluetoothobs_exists = False
gnssobs_exists = False
imuobs_exists = False
adjacences_exists = False


if path.exists('../Datasets/converted_datasets/' + dataset_name + '/devices.csv'):
    devices_frame = pd.read_csv('../Datasets/converted_datasets/' + dataset_name + '/devices.csv', na_values = 'NULL', sep=',', dtype=object)
    devices_exists = True

if path.exists('../Datasets/converted_datasets/' + dataset_name + '/users.csv'):
    users_frame = pd.read_csv('../Datasets/converted_datasets/' + dataset_name + '/users.csv', na_values = 'NULL', sep=',', dtype=object)
    users_exists = True

if path.exists('../Datasets/converted_datasets/' + dataset_name + '/tessellations.csv'):
    tessellations_frame = pd.read_csv('../Datasets/converted_datasets/' + dataset_name + '/tessellations.csv', na_values = 'NULL', sep=',', dtype=object)
    tessellations_exists = True

if path.exists('../Datasets/converted_datasets/' + dataset_name + '/wifi_obs.csv'):   
    wifi_obs_frame = pd.read_csv('../Datasets/converted_datasets/' + dataset_name + '/wifi_obs.csv', na_values = 'NULL', sep=',', dtype=object)
    wifiobs_exists = True

if path.exists('../Datasets/converted_datasets/' + dataset_name + '/blue_obs.csv'):   
    bluetooth_obs_frame = pd.read_csv('../Datasets/converted_datasets/' + dataset_name + '/blue_obs.csv', na_values = 'NULL', sep=',', dtype=object)
    bluetoothobs_exists = True
    
if path.exists('../Datasets/converted_datasets/' + dataset_name + '/gnss_obs.csv'):   
    gnss_obs_frame = pd.read_csv('../Datasets/converted_datasets/' + dataset_name + '/gnss_obs.csv', na_values = 'NULL', sep=',', dtype=object)
    gnssobs_exists = True
    
if path.exists('../Datasets/converted_datasets/' + dataset_name + '/imu_obs.csv'):   
    imu_obs_frame = pd.read_csv('../Datasets/converted_datasets/' + dataset_name + '/imu_obs.csv', na_values = 'NULL', sep=',', dtype=object)
    imuobs_exists = True
    
if path.exists('../Datasets/converted_datasets/' + dataset_name + '/adjacences.csv'):   
    adjacences_frame = pd.read_csv('../Datasets/converted_datasets/' + dataset_name + '/adjacences.csv', na_values = 'NULL', sep=',', dtype=object)
    adjacences_exists = True
    
    

assert fingerprints_exists and places_exists, "ERROR: missing fingeprints or places mandatory file"
assert not(adjacences_exists and not(places_exists)), "ERROR: missing places file but adjacences file specified"



# To be used to format possible null values in INSERT queries
def format_nan_string(value):
    if pd.isna(value):
        return 'null'
    else:
        return "'" + str(value) + "'"
    
    
# To be used to format possible null values in COPY FROM queries
def format_nan_string_csv(value):
    if pd.isna(value):
        return 'null'
    else:
        return str(value) 
    
    
# To be used to copy tuples from a temporary CSV file to a desination table
# Filename is the name of the temporary file, including its extension
# Table specification is the name of the table together with the destination attributes, as in an INSERT INTO query
def copy_to_table(filename, table_specification):
    cursor = connection.cursor()
    with open('./' + filename, 'r+') as f:
        cursor.copy_expert("COPY " + table_specification + "FROM STDIN WITH NULL AS 'null' DELIMITER ','", f)
    connection.commit() 
    f.close()    
    
    

In [3]:
try:
    # Connect to an existing database
    connection = psycopg2.connect(user="DBUSER",
                                  password="DBUSER_PASSWORD",
                                  host="158.110.145.70",
                                  port="5432", 
                                  database="Open_Fingerprinting",
                                  connect_timeout=3)

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
   
   
   
 
# Also, open SSH connection to the server, which will be used to transfer temporary CSV files
ssh = SSHClient()
ssh.set_missing_host_key_policy(AutoAddPolicy())
ssh.load_system_host_keys()
ssh.connect(hostname='158.110.145.70', 
            port = '4242',
            username='SERVERUSER',
            key_filename='./indoor',
            password='SHH_KEY_PWD')
ftp_client=ssh.open_sftp()

In [4]:
# Verify the consistency of loaded data: DEVICES.CSV

if devices_exists:

    print("Checking devices file...")

    # Are the columns of the file as expected?
    assert list(devices_frame.columns) == ['device_id', 'device_model', 'device_manufacturer', 'device_type', 'notes'], "ERROR: wrong columns in device file"

    # Are there replicated devices?
    tmp = np.unique(devices_frame['device_id'], return_counts=True)
    assert len(tmp[0][tmp[1] > 1]) == 0, "ERROR: replicated device IDs have been found, " + str(tmp[0][tmp[1] > 1])
    
    # Are there NULL values in device_id?
    assert (devices_frame['device_id'].isna()).values.sum() == 0, "ERROR: NULL values found in device_id oe"
    
    # Are there inconsistent device_model, device_manufacturer, and device_type occurrences wrt NULL values
    assert (devices_frame[['device_model', 'device_type', 'device_manufacturer']].isna()).values.sum() in [0,3], "ERROR: inconsistent presence of NULL values found in device_manufacturer, device_model, and/or device_type"


    # Are the device types not among those expected by the database?
    cursor = connection.cursor()
    cursor.execute("select description from device_model_type")
    record = [x[0] for x in cursor.fetchall()]
    connection.commit() 
    cursor.close()
    for raw_type in devices_frame['device_type'].unique():
        assert raw_type in record + [np.nan], "ERROR: Unexpected device type, " + str(raw_type)


    device_ids = np.unique(devices_frame['device_id'])


    print("   > OK.")

In [5]:
# Verify the consistency of loaded data: USERS.CSV

if users_exists:

    print("Checking users file...")

    # Are the columns of the file as expected?
    assert list(users_frame.columns) == ['user_id', 'username', 'type', 'notes'], "ERROR: wrong columns in users file"

    # Are there replicated users?
    tmp = np.unique(users_frame['user_id'], return_counts=True)
    assert len(tmp[0][tmp[1] > 1]) == 0, "ERROR: replicated user ids have been found, " + str(tmp[0][tmp[1] > 1])
    
    tmp = np.unique(users_frame['username'], return_counts=True)
    assert len(tmp[0][tmp[1] > 1]) == 0, "ERROR: replicated usernames have been found, " + str(tmp[0][tmp[1] > 1])
    
    
    # Are there unexpected types of users?
    distinct_types = np.unique(users_frame['type'])
    for dist_type in distinct_types:
        assert dist_type in ('online', 'trusted'), "ERROR: unexpected user type"

        
    # Are there NULL values in device_type or device_id?
    assert (users_frame[['user_id', 'type']].isna()).values.sum() == 0, "ERROR: NULL values found in either user_id or type"


    user_ids = np.unique(users_frame['user_id'])

    print("   > OK.")

In [6]:
# Verify the consistency of loaded data: PLACES.CSV

if places_exists:

    print("Checking places file...")

    # Are the columns of the file as expected?
    assert list(places_frame.columns) == ['building', 'floor', 'floor_number', 'site', 'site_height', 'site_area', 'floor_height', 'floor_area', 'building_area'], "ERROR: wrong columns in places file"

    # Are there replicated entries?
    assert len(places_frame.drop_duplicates()) == len(places_frame), "ERROR: replicated entries in places file have been found"  
    
    # Checking information consistency
    assert places_frame.groupby(['building']).building_area.nunique().max() <= 1, "ERROR: building with different areas found"
    assert places_frame.groupby(['building', 'floor'], dropna=False).floor_number.nunique().max() <= 1, "ERROR: floor with different floor numbers found"
    assert places_frame.groupby(['building', 'floor'], dropna=False).floor_height.nunique().max() <= 1, "ERROR: floor with different heights found"
    assert places_frame.groupby(['building', 'floor'], dropna=False).floor_area.nunique().max() <= 1, "ERROR: floor with different areas found"
    assert places_frame.groupby(['building', 'floor', 'site'], dropna=False).site_height.nunique().max() <= 1, "ERROR: site with different heights found"
    assert places_frame.groupby(['building', 'floor', 'site'], dropna=False).site_area.nunique().max() <= 1, "ERROR: site with different areas found"
    
    # Are there inconsistencies related to the NULL values?
    for ind, row in places_frame.iterrows():
        is_null = row.isna()
        if not(is_null['site']):
            assert not(is_null['floor']) and not(is_null['building']), "ERROR: non NULL site with NULL floor or building on row " + str(ind)
        if not(is_null['floor']):
            assert not(is_null['floor_number']), "ERROR: non NULL floor with NULL floor number on row " + str(ind)
            assert not(is_null['building']), "ERROR: non NULL floor with NULL building on row " + str(ind)
        if not(is_null['site_height']) or not(is_null['site_area']):
            assert not(is_null['site']), "ERROR: non NULL site info with NULL site " + str(ind)
        if not(is_null['floor_height']) or not(is_null['floor_area']):
            assert not(is_null['floor']), "ERROR: non NULL floor info with NULL floor " + str(ind)
        assert not(is_null['building']), "ERROR: NULL building found on row " + str(ind)
        
        int(row['floor_number']) # this checks whether the floor_number can be parsed to an integer

    # Aux variables needed later
    distinct_buildings = [str(x) for x in list(places_frame[['building']].drop_duplicates().values.flatten())]
    distinct_floors = [str(x[0]) + "_" + str(x[1]) for x in list(places_frame[['building', 'floor']].drop_duplicates().values)]
    distinct_sites = [str(x[0]) + "_" + str(x[1]) + "_" + str(x[2]) for x in list(places_frame[['building', 'floor', 'site']].drop_duplicates().values)]

    print("   > OK.")

Checking places file...
   > OK.


In [7]:
# Verify the consistency of loaded data: TESSELLATIONS.CSV

if tessellations_exists:

    print("Checking tessellations file...")

    # Are the columns of the file as expected?
    assert list(tessellations_frame.columns) == ['building', 'floor', 'site', 'tile', 'tessellation_type', 'coord_a_x', 'coord_a_y', 'coord_b_x', 'coord_b_y', 'coord_c_x', 'coord_c_y', 'coord_d_x', 'coord_d_y'], "ERROR: wrong columns in tessellations file"

    # Are there replicated entries?
    assert len(tessellations_frame.drop_duplicates()) == len(tessellations_frame), "ERROR: replicated entries in tessellations file have been found"

    # Are there replicated tile_ids?
    assert len(tessellations_frame[['building', 'floor', 'site', 'tile']].drop_duplicates()) == len(tessellations_frame), "ERROR: replicated tile names (within the same floor) in tessellations file have been found"


    tile_id_map_type = {}
    # Are there inconsistencies in the rows?
    pbar = tqdm(total=len(tessellations_frame))
    for ind, row in tessellations_frame.iterrows():
        is_null = row.isna()

        tile_id_map_type[str(row['building']) + '_' + str(row['floor']) + '_' + str(row['site']) + '_' + str(row['tile'])] = row['tessellation_type']

        assert not(is_null['building']), "ERROR: NULL building found, row " + str(ind)
        assert not(is_null['floor']), "ERROR: NULL floor found, row " + str(ind)
        assert not(is_null['tile']), "ERROR: NULL tile found, row " + str(ind)
        assert not(is_null['tessellation_type']), "ERROR: NULL tessellation_type found, row " + str(ind)
        assert row['tessellation_type'] in ['logical', 'zone', 'grid', 'crowd'], "ERROR: unexpected tessellation_type, row " + str(ind)

        # Checks for the logical tiles: each must be related to a site or floor, and at most coord_a_x and coord_a_y may be given a value 
        if row['tessellation_type'] == 'logical':
            assert is_null[7:].sum() == 6 and (is_null[5:7].sum() in [0, 2]), "ERROR: wrong set of coordinates given for a logical tile, row " + str(ind)
            assert np.sum(is_null[['building', 'floor', 'site']]) == 0 or np.sum(is_null[['building', 'floor']]) == 0, "ERROR: logical tile not linked to non NULL building, floor, (possibly) site. Row " + str(ind)
            assert str(row['building']) + "_" + str(row['floor'])  + "_" + str(row['site']) in  distinct_sites, "ERROR: unexpected site encountered with respect to places file, row " + str(ind)

        # Checks for the zone tiles: each must be related to a site or floor, and all coord_*_x and coord_*_y must be given a value  
        if row['tessellation_type'] == 'zone':
            assert is_null[5:].sum() == 0, "ERROR: wrong set of coordinates given for a zone tile, row " + str(ind)
            assert np.sum(is_null[['building', 'floor', 'site']]) == 0 or np.sum(is_null[['building', 'floor']]) == 0, "ERROR: zone tile not linked to non NULL building, floor, (possibly) site. Row " + str(ind)
            assert str(row['building']) + "_" + str(row['floor'])  + "_" + str(row['site']) in  distinct_sites, "ERROR: unexpected site encountered with respect to places file, row " + str(ind)

        # Checks for the grid tiles: each must be related to a floor, and all coord_*_x and coord_*_y must be given a value 
        if row['tessellation_type'] == 'grid':
            assert is_null[5:].sum() == 0, "ERROR: wrong set of coordinates given for a grid tile, row " + str(ind)
            assert np.sum(is_null[['building', 'floor']]) == 0 and is_null['site'], "ERROR: grid tile not correctly linked to a floor and building, row " + str(ind)
            assert str(row['building']) + "_" + str(row['floor'])  + "_" + str(row['site']) in distinct_sites or str(row['building']) + "_" + str(row['floor']) in distinct_floors, "ERROR: unexpected floor encountered with respect to places file, row " + str(ind)
            
        # Checks for the crowd tiles:
        if row['tessellation_type'] == 'crowd':
            # each must be related to a floor, and not have any coordinates
            assert is_null[5:].sum() > 0, "ERROR: wrong set of coordinates given for a crowd tile, row " + str(ind)
            assert np.sum(is_null[['building', 'floor']]) == 0 and is_null['site'], "ERROR: grid tile not correctly linked to a floor and building, row " + str(ind)
            assert str(row['building']) + "_" + str(row['floor'])  + "_" + str(row['site']) in distinct_sites or str(row['building']) + "_" + str(row['floor']) in distinct_floors, "ERROR: unexpected floor encountered with respect to places file, row " + str(ind)
        
        pbar.update(1)
    pbar.close()
            
    # other check for crowd tessellations: a floor may contain at most one crowd tile
    tmpcrd = tessellations_frame[tessellations_frame['tessellation_type'] == 'crowd'][['building', 'floor', 'tile']].groupby(['building', 'floor'], dropna=False).count()['tile'].values
    if len(tmpcrd) > 1:
        assert tmpcrd.max() == 1, "ERROR: floor encountered with more than one crowd tile"
        

    print("   > OK.")

100%|██████████| 3/3 [00:00<00:00, 322.95it/s]

Checking tessellations file...
   > OK.


In [8]:
# Verify the consistency of loaded data: FINGERPRINTS.CSV

if fingerprints_exists:

    print("Checking fingerprints file...")

    # Are the columns of the file as expected?
    assert list(fingerprints_frame.columns) == ['fingerprint_id', 'coord_x', 'coord_y', 'coord_z', 'building', 'floor', 'site', 'tile', 'user_id', 'device_id', 'epoch', 'set', 'is_radio_map', 'preceded_by', 'followed_by', 'notes']

    # Are there replicated entries?
    assert len(fingerprints_frame.drop_duplicates()) == len(fingerprints_frame), "ERROR: replicated entries in fingerprints file have been found"

    # Are there replicated fingerprint_ids?
    assert len(fingerprints_frame['fingerprint_id'].drop_duplicates()) == len(fingerprints_frame), "ERROR: replicated fingerprint_ids in fingerprints file have been found"


    # Are there inconsistencies in the rows?
    pbar = tqdm(total=len(fingerprints_frame))
    for ind, row in fingerprints_frame.iterrows():
        is_null = row.isna()

        assert not(is_null[0]), "ERROR: NULL value in fingerprint_id, row " + str(ind)
        assert np.sum(is_null[1:3]) in [0,2] and np.sum(is_null[1:4]) in [0,1,3], "ERROR: wrong set of coordinates given for a fingeprint, row " + str(ind)
        assert is_null['device_id'] or row['device_id'] in device_ids, "ERROR: unexpected device_id found, row " + str(ind)
        assert is_null['user_id'] or row['user_id'] in user_ids, "ERROR: unexpected user_id found, row " + str(ind)
        assert is_null['set'] or row['set'] in ['training', 'validation', 'test'], "ERROR: unexpected value for set, row " + str(ind)
        assert not is_null['is_radio_map'], "ERROR: unspecified is_radio_map value for a given fingerprint, row " + str(ind)
        #assert not is_null['epoch'], "ERROR: unspecified epoch value for a given fingerprint, row " + str(ind)

        if not(is_null['site']):
            assert not(is_null['floor']) and not(is_null['building']), "ERROR: non NULL site with NULL floor or building, row " + str(ind)
            assert str(row['building']) + "_" + str(row['floor'])  + "_" + str(row['site']) in  distinct_sites, "ERROR: unexpected site encountered with respect to places file, row " + str(ind)
        if not(is_null['floor']):
            assert not(is_null['building']), "ERROR: non NULL floor with NULL building, row " + str(ind)
            assert str(row['building']) + "_" + str(row['floor']) in distinct_floors, "ERROR: unexpected floor encountered with respect to places file, row " + str(ind)
        if not(is_null['building']):
            assert str(row['building']) in distinct_buildings, "ERROR: unexpected building encountered with respect to places file, row " + str(ind)
        if row['is_radio_map'] == 'True':
            assert not(is_null['tile']), "ERROR: radio map fingerprint without a tile, row " + str(ind) 
        if not(is_null['preceded_by']):
            assert row['preceded_by'] in fingerprints_frame['fingerprint_id'].values, "ERROR: could not find any references related to the preceding fingerprint id, row " + str(ind)
        if not(is_null['followed_by']):
            assert row['followed_by'] in fingerprints_frame['fingerprint_id'].values, "ERROR: could not find any references related to the following fingerprint id, row " + str(ind)
            
        # Checking that tile is used in a consistent way with respect to the data defined in the file TESSELLATIONS.CSV
        if not(is_null['tile']):
            tile_kind = tile_id_map_type[str(row['building']) + '_' + str(row['floor']) + '_' + str(row['site']) + '_' + str(row['tile'])]
            
            if tile_kind in ['logical', 'zone']:
                assert not(is_null['floor']), "ERROR: logical or zone tile without a floor, row " + str(ind)
            
            if tile_kind in ['grid', 'crowd']:
                assert not(is_null['floor']), "ERROR: grid or crowd tile without a floor, row " + str(ind)
                assert is_null['site'], "ERROR: grid or crowd tile with a site, row " + str(ind)
                
        pbar.update(1)
    pbar.close()
                
                
    print("   > OK.")
    

  1%|          | 71/9291 [00:00<00:13, 705.33it/s]

Checking fingerprints file...


100%|██████████| 9291/9291 [00:11<00:00, 820.39it/s] 

   > OK.


In [9]:
# Verify the consistency of loaded data: ADJACENCES.CSV


def test_if_in_map(row):
    assert str(row['tile_1_building']) + "_" + str(row['tile_1_floor'])  + "_" + str(row['tile_1_site']) + "_" + str(row['tile_1_tile']) in  tile_id_map_type, "ERROR: unexpected tile encountered with respect to tessellations file, row " + str(ind)
    


non_existing = []

if adjacences_exists:
    
    print("Checking adjacences file...")

    # Are the columns of the file as expected?
    assert list(adjacences_frame.columns) == ['tile_1_building', 'tile_1_floor', 'tile_1_site', 'tile_1_tile', 'tile_2_building', 'tile_2_floor', 'tile_2_site', 'tile_2_tile', 'walkable', 'cost']

    # Are there NULL values in the file?
    assert adjacences_frame[['tile_1_building', 'tile_1_floor', 'tile_1_tile', 'tile_2_building', 'tile_2_floor', 'tile_2_tile']].isna().sum().sum() == 0, "ERROR: null values in the adjacences file"
    
    # Are there replicated entries?
    assert len(adjacences_frame.drop_duplicates()) == len(adjacences_frame), "ERROR: replicated entries in adjacences file have been found"

    # Are there inconsistencies in the rows?
    def test_if_in_map(row):
        assert str(row['tile_1_building']) + "_" + str(row['tile_1_floor'])  + "_" + str(row['tile_1_site']) + "_" + str(row['tile_1_tile']) in  tile_id_map_type, "ERROR: unexpected tile encountered with respect to tessellations file, row " + str(ind)

    _ = adjacences_frame.apply(lambda x: test_if_in_map(x), axis=1)
    
    
    # Check if symmetries are satisfied
    # Here, it makes sense to reason independently on each floor, since connections between different floors may not be symmetric (e.g., escalator)
    for bld in fingerprints_frame['building'].drop_duplicates():
        for flr in fingerprints_frame['floor'].drop_duplicates():
            print('   ' + str(bld) + '   ' + str(flr))
            # Filtering the adjacencies to keep those on the specific floor
            adjs_red = adjacences_frame[(adjacences_frame['tile_1_building'] == bld) & (adjacences_frame['tile_1_floor'] == flr) & (adjacences_frame['tile_2_building'] == bld) & (adjacences_frame['tile_2_floor'] == flr)]
            
            if len(adjs_red) == 0:
                continue
            
            num_rps = len(adjs_red[['tile_1_building', 'tile_1_floor', 'tile_1_site', 'tile_1_tile']].drop_duplicates())
            aux_matrix = np.zeros((num_rps, num_rps, 2))
                        
            # first dimension is used to keep track of the connections (sum 1 at each observed one, hoping to obtain a symmetric matrix composed on 1s and 0s)
            # second dimension is used in the same way but for the walkability requirement
            # cell (i,j) tells us information regarding the link going from "i" to "j"

            # first thing we have to assign a unique index to each tile
            # let us look at the first half of the columns to do that
            # we expect to build a map having "num_rps" elements
            
            tile_map_idx_matr = {}
            tmp_idx = 0
            for ind, row in adjs_red.iterrows():
                if str(row.tile_1_building) + "_" + str(row.tile_1_floor) + "_" + str(row.tile_1_site) + "_" + str(row.tile_1_tile) not in tile_map_idx_matr:
                    tile_map_idx_matr[str(row.tile_1_building) + "_" + str(row.tile_1_floor) + "_" + str(row.tile_1_site) + "_" + str(row.tile_1_tile)] = tmp_idx
                    tmp_idx += 1

            # now we populate "aux_matrix"
            for ind, row in adjs_red.iterrows():
                i_index = tile_map_idx_matr[str(row.tile_1_building) + "_" + str(row.tile_1_floor) + "_" + str(row.tile_1_site) + "_" + str(row.tile_1_tile)]
                j_index = tile_map_idx_matr[str(row.tile_2_building) + "_" + str(row.tile_2_floor) + "_" + str(row.tile_2_site) + "_" + str(row.tile_2_tile)]
                aux_matrix[i_index, j_index, 0] += 1
                aux_matrix[i_index, j_index, 1] += int(row.walkable)



            # are there repeated connections?
            assert np.max(aux_matrix) == 1, "Repeated connections found"



            #https://stackoverflow.com/questions/42908334/checking-if-a-matrix-is-symmetric-in-numpy
            import scipy
            def is_symmetric(A, tol=1e-8):
                return scipy.linalg.norm(A-A.T, scipy.Inf) < tol;


            # is the matrix symmetric WRT the connections?
            check_symm_matr = aux_matrix[:, :, 0] - aux_matrix[:, :, 0].T
            if np.max(check_symm_matr) > 0:
                rows, cols = np.where(check_symm_matr > 0)
                reverse_map = {}
                for x in tile_map_idx_matr.keys():
                    reverse_map[tile_map_idx_matr[x]] = x
                for i in range(len(rows)):
                    print(reverse_map[rows[i]] + "->" + reverse_map[cols[i]])

                assert False, "Non-symmetric connection matrix"


            # is the matrix symmetric WRT walkability?
            check_symm_matr = aux_matrix[:, :, 1] - aux_matrix[:, :, 1].T
            if np.max(check_symm_matr) > 0:
                rows, cols = np.where(check_symm_matr > 0)
                reverse_map = {}
                for x in tile_map_idx_matr.keys():
                    reverse_map[tile_map_idx_matr[x]] = x
                for i in range(len(rows)):
                    print(reverse_map[rows[i]] + "->" + reverse_map[cols[i]])

                assert False, "Non-symmetric walkability matrix"



    print("   > OK.")
    

In [10]:
# Verify the consistency of loaded data: WIFI_OBS.CSV

import re

if wifiobs_exists:

    print("Checking wifi observations file...")
 
    # Are the columns of the file as expected?
    assert list(wifi_obs_frame.columns)[0] == 'fingerprint_id', 'ERROR: wrong columns specified in wifi observations file'
    assert np.any([len(re.findall("^AP-.+-.+", x)) == 1 for x in list(wifi_obs_frame.columns)[1:]]), 'ERROR: wrong columns specified in wifi observations file'

    assert len(wifi_obs_frame['fingerprint_id'].drop_duplicates()) == len(wifi_obs_frame), "ERROR: replicated fingerprint_ids in wifi observations file have been found"

    assert np.max(wifi_obs_frame.iloc[:,1:].fillna(-1).values.astype('float')) <= 0, "ERROR: wrong RSS specified in wifi observations file"
    
    print("   > OK.")   

Checking wifi observations file...
   > OK.


In [11]:
# Verify the consistency of loaded data: BLUE_OBS.CSV

import re

if bluetoothobs_exists:

    print("Checking bluetooth observations file...")
 
    # Are the columns of the file as expected?
    assert list(bluetooth_obs_frame.columns)[0] == 'fingerprint_id', 'ERROR: wrong columns specified in bluetooth observations file'
    assert np.any([len(re.findall("^BL-.+-.+", x)) == 1 for x in list(bluetooth_obs_frame.columns)[1:]]), 'ERROR: wrong columns specified in bluetooth observations file'

    assert len(bluetooth_obs_frame['fingerprint_id'].drop_duplicates()) == len(bluetooth_obs_frame), "ERROR: replicated fingerprint_ids in bluetooth observations file have been found"

    assert np.max(bluetooth_obs_frame.iloc[:,1:].fillna(-1).values.astype('float')) <= 0, "ERROR: wrong RSS specified in bluetooth observations file"
    
    print("   > OK.")  

In [12]:
# Verify the consistency of loaded data: GNSS_OBS.CSV

if gnssobs_exists:

    print("Checking gnss observations file...")
 
    # Are the columns of the file as expected?
    assert list(gnss_obs_frame.columns) == ['fingerprint_id', 'latitude', 'longitude', 'elevation', 'num_satellites'], 'ERROR: wrong columns specified in gnss observations file'

    assert len(gnss_obs_frame['fingerprint_id'].drop_duplicates()) == len(gnss_obs_frame), "ERROR: replicated fingerprint_ids in gnss observations file have been found"

    # Are there NULL values in the file?
    assert gnss_obs_frame.isna().sum().sum() == 0, "ERROR: null values in the gnss observations file"
    
    print("   > OK.")  

In [13]:
# Verify the consistency of loaded data: IMU_OBS.CSV

if imuobs_exists:

    print("Checking IMU observations file...")
 
    # Are the columns of the file as expected?
    assert list(imu_obs_frame.columns) == ['fingerprint_id', 'epoch', 'acc_x', 'acc_y', 'acc_z', 'mag_x', 'mag_y', 'mag_z', 'gyr_x', 'gyr_y', 'gyr_z'], 'ERROR: wrong columns specified in IMU observations file'

    print("   > OK.")  

In [14]:
########################################################################## NOW WE HAVE THE IMPORT PROCESS ##########################################################################

In [15]:
# Clean the data staging area before beginning the import process

cursor = connection.cursor()
cursor.execute("select data_staging.truncate_tables('data_staging')")
record = cursor.fetchall()
connection.commit() 
cursor.close()

In [16]:
# Keep track of all the IDs that you are importing

cursor = connection.cursor()
cursor.execute("select current_timestamp::text")
record = cursor.fetchall()
connection.commit() 
cursor.close()
import_timestamp = record[0][0]



file_imported = open('./temp_imported.csv', 'w')

In [17]:
# Data source 
cursor = connection.cursor()
string_insert = "INSERT INTO data_staging.data_source(name, url, notes) VALUES ('" + str(dataset_name) + "', " + str(format_nan_string(dataset_url)) + ", " + str(format_nan_string(dataset_notes)) + ")"
string_insert = string_insert.replace("'NULL'", "null")
cursor.execute(string_insert)
connection.commit() 
cursor.close()

cursor = connection.cursor()
cursor.execute("select id from data_staging.data_source")
dsid = cursor.fetchall()[0][0]
connection.commit() 
cursor.close()

file_imported.write(str(dsid) + "," + str(import_timestamp) + "," + str(dataset_name) + "," + "data_source\n")

# Is there another data source with the same name or url already stored in the database?

cursor = connection.cursor()
cursor.execute("select name, url from public.data_source")
res = cursor.fetchall()
connection.commit() 
cursor.close()

if len(res) > 0:
    #if dataset_url != 'NULL':
    #    assert not(dataset_name in np.asarray(res)[:,0] or dataset_url in np.asarray(res)[:,1]), "ERROR: dataset appears to be already present in the database public schema"
   # else:
        assert not(dataset_name in np.asarray(res)[:,0]), "ERROR: dataset appears to be already present in the database public schema"
        

In [18]:
# Devices

if devices_exists:
    
    unique_devices = devices_frame[['device_model', 'device_manufacturer', 'device_type']].drop_duplicates()

        
    # Insert into device_model
    for _, dev in unique_devices.iterrows():
        if np.sum(dev.isna()) == 0:
            # To insert in case which the device moodel does not exist
            string_insert = "INSERT INTO data_staging.device_model(name, manufacturer, type_id) \
                                SELECT '" + str(dev[0]) + "','" + str(dev[1]) + "', device_model_type.id \
                                FROM device_model_type \
                                WHERE device_model_type.description = '" + str(dev[2]) + "'\
                                        AND NOT EXISTS (SELECT * \
                                                        FROM public.device_model \
                                                        WHERE device_model.name = '" + str(dev[0]) + "' and device_model.manufacturer = '" + str(dev[1]) + "');"
            cursor = connection.cursor()
            cursor.execute(string_insert)
            connection.commit() 
            cursor.close()




    # Generating new ids for the single devices
    cursor = connection.cursor()
    cursor.execute("truncate table data_staging.generated_ids;")
    connection.commit() 
    cursor.close()

    cursor = connection.cursor()
    string_insert = "INSERT INTO data_staging.generated_ids(dummy_field) VALUES "
    for b_id in np.unique(devices_frame['device_id']):
        string_insert += "(null), "
    string_insert = string_insert[:-2] + ";"
    cursor.execute(string_insert)
    connection.commit() 
    cursor.close()

    cursor = connection.cursor()
    cursor.execute("select id from data_staging.generated_ids")
    generated_ids = [x[0] for x in cursor.fetchall()]
    connection.commit() 
    cursor.close()



    # Inserting the single devices
    device_code_map_id = {} # maps that keeps the correspondence between the device code in the dataset and its newly generated ID in the database
    for idx, ind_row in enumerate(devices_frame.iterrows()):
        row = ind_row[1]
        device_code_map_id[row.device_id] = generated_ids[idx]
        string_insert = "INSERT INTO data_staging.device(code, data_source_id, id, device_model_id, notes) \
                            SELECT '" + str(row.device_id)  + "', data_source.id, " + str(generated_ids[idx]) + ", COALESCE(ds_model.id, pb_model.id), " + format_nan_string(row.notes) + "   \
                            FROM data_staging.data_source \
                                    LEFT OUTER JOIN data_staging.device_model AS ds_model ON ds_model.manufacturer = '" + str(row.device_manufacturer) + "' AND ds_model.name = '" + str(row.device_model) + "'\
                                    LEFT OUTER JOIN device_model_type AS ds_model_type ON ds_model.type_id = ds_model_type.id AND ds_model_type.description = '" + str(row.device_type) + "'\
                                    LEFT OUTER JOIN device_model AS pb_model ON pb_model.manufacturer = '" + str(row.device_manufacturer) + "' AND pb_model.name = '" + str(row.device_model) + "'\
                                    LEFT OUTER JOIN device_model_type AS pb_model_type ON pb_model.type_id = pb_model_type.id AND pb_model_type.description = '" + str(row.device_type) + "';" 
        cursor = connection.cursor()
        cursor.execute(string_insert)
        connection.commit() 
        cursor.close()
        
        
    for insid in generated_ids:
        file_imported.write(str(insid) + "," + str(import_timestamp) + "," + str(dataset_name) + "," + "device\n")


In [19]:
# Users


if users_exists:

    # Generating new ids for the single users
    cursor = connection.cursor()
    cursor.execute("truncate table data_staging.generated_ids;")
    connection.commit() 
    cursor.close()

    cursor = connection.cursor()
    string_insert = "INSERT INTO data_staging.generated_ids(dummy_field) VALUES "
    for i in range(len(users_frame)):
        string_insert += "(null), "
    string_insert = string_insert[:-2] + ";"
    cursor.execute(string_insert)
    connection.commit() 
    cursor.close()

    cursor = connection.cursor()
    cursor.execute("select id from data_staging.generated_ids")
    generated_ids = [x[0] for x in cursor.fetchall()]
    connection.commit() 
    cursor.close()



    # Inserting the single users
    user_code_map_id = {} # maps that keeps the correspondence between the user code in the dataset and its newly generated ID in the database
    for idx, indrow in enumerate(users_frame.iterrows()):
        row = indrow[1]
        user_code_map_id[row.user_id] = generated_ids[idx]
        cursor = connection.cursor()
        if row.type == 'online':
            u_type = 1
        else:
            u_type = 2
        
        string_insert = "INSERT INTO data_staging.user(id, code, username, type_id, data_source_id, notes) \
                        SELECT " + str(generated_ids[idx]) + ", '" + str(row.user_id) + "', " + format_nan_string(row.username) + ", " + str(u_type) + ", data_source.id, " + format_nan_string(row.notes) + "\
                        FROM data_staging.data_source;"
        string_insert = string_insert.replace("'NULL'", "null")
        
        cursor.execute(string_insert)
        connection.commit() 
        cursor.close()

        
    for insid in generated_ids:
        file_imported.write(str(insid) + "," + str(import_timestamp) + "," + str(dataset_name) + "," + "user\n")


In [20]:
# Building (+ Place, + place_data_source)

# Generating new ids
cursor = connection.cursor()
cursor.execute("truncate table data_staging.generated_ids;")
connection.commit() 
cursor.close()

cursor = connection.cursor()
string_insert = "INSERT INTO data_staging.generated_ids(dummy_field) VALUES "
for b_id in np.unique(places_frame['building']):
    string_insert += "(null), "
string_insert = string_insert[:-2] + ";"
cursor.execute(string_insert)
connection.commit() 
cursor.close()

cursor = connection.cursor()
cursor.execute("select id from data_staging.generated_ids")
generated_ids = [x[0] for x in cursor.fetchall()]
connection.commit() 
cursor.close()


# Inserting into place
building_map_id = {} # maps that keeps the correspondence between the building code in the dataset and its newly generated ID in the database
cursor = connection.cursor()
string_insert = "INSERT INTO data_staging.place(id, name) VALUES "
for idx, b_id in enumerate(np.unique(places_frame['building'])):
    string_insert += "(" + str(generated_ids[idx]) + ",'" + str(b_id) + "'), "
    building_map_id[b_id] = generated_ids[idx]
string_insert = string_insert[:-2] + ";"
cursor.execute(string_insert)
connection.commit() 
cursor.close()                                

                                
# Inserting into building
cursor = connection.cursor()
string_insert = "INSERT INTO data_staging.building(place_id, area) VALUES "
for idx, bldinfo in enumerate(places_frame[['building', 'building_area']].drop_duplicates().iterrows()):
    string_insert += "(" + str(generated_ids[idx]) + "," + format_nan_string(bldinfo[1].building_area) + "), "
string_insert = string_insert[:-2] + ";"
cursor.execute(string_insert)
connection.commit() 
cursor.close()


# Inserting into place_data_source
cursor = connection.cursor()
string_insert = "INSERT INTO data_staging.place_data_source(data_source_id, place_id) \
                    SELECT data_source.id, building.place_id FROM data_staging.data_source JOIN data_staging.building ON 1=1;"
cursor.execute(string_insert)
connection.commit() 
cursor.close()


for insid in generated_ids:
    file_imported.write(str(insid) + "," + str(import_timestamp) + "," + str(dataset_name) + "," + "building\n")

In [21]:
# Floor (+ Place, + place_data_source, + contains) including floor above and below relationships

bld_flr_names = places_frame[np.logical_not(pd.isna(places_frame.floor))][['building', 'floor', 'floor_number', 'floor_height', 'floor_area']].drop_duplicates()


if len(bld_flr_names) > 0: # if there are some floors to import

    # Generating new ids
    cursor = connection.cursor()
    cursor.execute("truncate table data_staging.generated_ids;")
    connection.commit() 
    cursor.close()

    cursor = connection.cursor()
    string_insert = "INSERT INTO data_staging.generated_ids(dummy_field) VALUES "
    for i in range(len(bld_flr_names)):
        string_insert += "(null), "
    string_insert = string_insert[:-2] + ";"
    cursor.execute(string_insert)
    connection.commit() 
    cursor.close()

    cursor = connection.cursor()
    cursor.execute("select id from data_staging.generated_ids")
    generated_ids = [x[0] for x in cursor.fetchall()]
    connection.commit() 
    cursor.close()





    bldflr_map_id = {} # maps that keeps the correspondence between the "floor_building" code in the dataset and its newly generated ID in the database
    bldflr_map_number = {} # maps that tells us the number (vertical ordering) of each floor
    for idx, bld_flr in enumerate(bld_flr_names.iterrows()):
        bld = bld_flr[1].building
        flr = bld_flr[1].floor
        floor_height = bld_flr[1].floor_height
        floor_area = bld_flr[1].floor_area
        bldflr_map_id[str(bld) + "_" + str(flr)] = generated_ids[idx]
        bldflr_map_number[str(bld) + "_" + str(flr)] = bld_flr[1].floor_number

        # Insert into place
        cursor = connection.cursor()
        cursor.execute("INSERT INTO data_staging.place(id, name) VALUES (" + str(generated_ids[idx]) + ",'" + str(flr) + "');")
        connection.commit() 
        cursor.close()

        # Insert into place_data_source
        cursor = connection.cursor()
        string_insert = "INSERT INTO data_staging.place_data_source(data_source_id, place_id) SELECT id, " + str(generated_ids[idx]) + " from data_staging.data_source;"
        cursor.execute(string_insert)
        connection.commit() 
        cursor.close()

        # Update contains
        cursor = connection.cursor()
        string_insert = "INSERT INTO data_staging.contains(container_place_id, contained_place_id) VALUES (" + str(building_map_id[bld]) + "," + str(generated_ids[idx]) + ");"
        cursor.execute(string_insert)
        connection.commit() 
        cursor.close()    

        # Insert into floor
        cursor = connection.cursor()
        string_insert = "INSERT INTO data_staging.floor(place_id, area, height) VALUES (" + str(generated_ids[idx]) + "," + format_nan_string(floor_area) + "," + format_nan_string(floor_height) + ");"
        cursor.execute(string_insert)
        connection.commit() 
        cursor.close()


    # Update above/below floor relationships

    cursor = connection.cursor()
    cursor.execute("select bldplace.name as bldname, flrplace.name as flrname, flrplace.id as flrid \
                    from data_staging.place as bldplace \
                            join data_staging.building on building.place_id = bldplace.id \
                            join data_staging.contains on container_place_id = building.place_id  \
                            join data_staging.place as flrplace on flrplace.id = contained_place_id \
                    order by flrplace.name::text, bldplace.name::text;")
    records = np.asarray(cursor.fetchall()) # building name, floor name, floor id
    connection.commit() 
    cursor.close()    


    bld_flrnumber_map_flr = {} # given a building name and a floor number, it gives me the floor name
    for key in bldflr_map_number.keys():
        bld = key.split("_")[0]
        flr = key.split("_")[1]
        num = bldflr_map_number[key]
        bld_flrnumber_map_flr[bld + "_" + str(num)] = str(flr)


    for bldname in np.unique(records[:,0]):
        records_bld = records[records[:,0] == bldname]
        if len(records_bld) > 1: # if there is more than one floor in the considered building
            bld_floor_numbers = np.unique([int(bldflr_map_number[str(records_bld[i, 0]) + "_" + str(records_bld[i, 1])]) for i in range(len(records_bld))])
            for bld_flr_flrid in records_bld:
                flr_number = int(bldflr_map_number[str(bld_flr_flrid[0] + "_" + bld_flr_flrid[1])])            
                if flr_number == bld_floor_numbers[0]:
                    flr_number_above = bld_floor_numbers[np.where(bld_floor_numbers > flr_number)[0][0]]
                    string_update = "UPDATE data_staging.floor SET below_of_floor_place_id = " + str(bldflr_map_id[bldname + "_" + bld_flrnumber_map_flr[str(bldname) + "_" + str(flr_number_above)]]) + " WHERE place_id = " +  str(bld_flr_flrid[2]) + ";"
                elif flr_number == bld_floor_numbers[-1]:
                    flr_number_below = bld_floor_numbers[np.where(bld_floor_numbers < flr_number)[0][-1]]
                    string_update = "UPDATE data_staging.floor SET above_of_floor_place_id = " + str(bldflr_map_id[bldname + "_" + bld_flrnumber_map_flr[str(bldname) + "_" + str(flr_number_below)]])  + " WHERE place_id = " +  str(bld_flr_flrid[2]) + ";"
                else:
                    flr_number_above = bld_floor_numbers[np.where(bld_floor_numbers > flr_number)[0][0]]
                    flr_number_below = bld_floor_numbers[np.where(bld_floor_numbers < flr_number)[0][-1]]
                    string_update = "UPDATE data_staging.floor SET below_of_floor_place_id = " + str(bldflr_map_id[bldname + "_" + bld_flrnumber_map_flr[str(bldname) + "_" + str(flr_number_above)]]) + \
                                    ", above_of_floor_place_id = " + str(bldflr_map_id[bldname + "_" + bld_flrnumber_map_flr[str(bldname) + "_" + str(flr_number_below)]]) + \
                                    " WHERE place_id = " +  str(bld_flr_flrid[2]) + ";"
                cursor = connection.cursor()
                cursor.execute(string_update)
                connection.commit() 
                cursor.close()    


    for insid in generated_ids:
        file_imported.write(str(insid) + "," + str(import_timestamp) + "," + str(dataset_name) + "," + "floor\n")

In [22]:
# Site (+ Place, + place_data_source, + contains) 

bld_flr_site_names = places_frame[(np.logical_not(pd.isna(places_frame.floor))) & (np.logical_not(pd.isna(places_frame.site)))][['building', 'floor', 'site', 'site_height', 'site_area']].drop_duplicates()



if len(bld_flr_site_names) > 0: # if there are some sites to import


    # Generating new ids
    cursor = connection.cursor()
    cursor.execute("truncate table data_staging.generated_ids;")
    connection.commit() 
    cursor.close()
    
    with open('./temp.csv', 'w') as file:
        for i in range(len(bld_flr_site_names)):
            file.write("null\n")
    copy_to_table("temp.csv", "data_staging.generated_ids(dummy_field)")

    cursor = connection.cursor()
    cursor.execute("select id from data_staging.generated_ids")
    generated_ids = [x[0] for x in cursor.fetchall()]
    connection.commit() 
    cursor.close()

    
        
    file_place = open('./temp_place.csv', 'w')
    file_place_data_source = open('./temp_place_data_source.csv', 'w')
    file_contains = open('./temp_contains.csv', 'w')
    file_site = open('./temp_site.csv', 'w')
    
    bldflrsite_map_id = {} # maps that keeps the correspondence between the "building_floor_site" code in the dataset and its newly generated ID in the database
    for idx, flr_bld_site in enumerate(bld_flr_site_names.iterrows()):
        bld = flr_bld_site[1].building
        flr = flr_bld_site[1].floor
        site = flr_bld_site[1].site
        site_height = flr_bld_site[1].site_height
        site_area = flr_bld_site[1].site_area

        bldflrsite_map_id[str(bld) + "_" + str(flr) + "_" + str(site)] = generated_ids[idx]


        # Insert into place
        file_place.write(str(generated_ids[idx]) + "," + str(site) + "\n")

        # Insert into place_data_source
        file_place_data_source.write(str(dsid) + "," + str(generated_ids[idx]) + "\n")


        # Insert into contains
        file_contains.write(str(bldflr_map_id[str(bld) + "_" + str(flr)]) + "," + str(generated_ids[idx]) + "\n")


        # Insert into site
        file_site.write(str(generated_ids[idx]) + "," + format_nan_string_csv(site_area) + "," + format_nan_string_csv(site_height) + "\n")
        
    
    file_place.close()
    file_place_data_source.close()
    file_contains.close()
    file_site.close()       

    copy_to_table("temp_place.csv", "data_staging.place(id, name)")   
    copy_to_table("temp_place_data_source.csv", "data_staging.place_data_source(data_source_id, place_id)") 
    copy_to_table("temp_contains.csv", "data_staging.contains(container_place_id, contained_place_id)") 
    copy_to_table("temp_site.csv", "data_staging.site(place_id, area, height)") 


    for insid in generated_ids:
        file_imported.write(str(insid) + "," + str(import_timestamp) + "," + str(dataset_name) + "," + "site\n")
        

In [23]:
# Tessellation

bldflrtype_map_tessid = {}
if tessellations_exists:
    
    distinct_tass = tessellations_frame[['building', 'floor', 'tessellation_type']].drop_duplicates()
    
    # Generating new ids
    cursor = connection.cursor()
    cursor.execute("truncate table data_staging.generated_ids;")
    connection.commit() 
    cursor.close()
    
    with open('./temp.csv', 'w') as file:
        for i in range(len(distinct_tass)):
            file.write("null\n")
    copy_to_table("temp.csv", "data_staging.generated_ids(dummy_field)")

    cursor = connection.cursor()
    cursor.execute("select id from data_staging.generated_ids")
    generated_ids = [x[0] for x in cursor.fetchall()]
    connection.commit() 
    cursor.close()
    
    

    with open('./temp.csv', 'w') as file:
        for ir, row in enumerate(distinct_tass.iterrows()):
            file.write(str(generated_ids[ir]) + "," + str(bldflr_map_id[str(row[1]['building']) + "_" + str(row[1]['floor'])]) + "," + str(dsid) + "," + str(row[1]['tessellation_type']) +"\n")
            bldflrtype_map_tessid[str(row[1]['building']) + "_" + str(row[1]['floor']) + "_" + str(row[1]['tessellation_type'])] = generated_ids[ir]
            
    copy_to_table("temp.csv", "data_staging.tessellation(id, floor_place_id, data_source_id, type)")        
            
        
    for insid in generated_ids:
        file_imported.write(str(insid) + "," + str(import_timestamp) + "," + str(dataset_name) + "," + "tessellation\n")


In [24]:
# Tile


if tessellations_exists:

    # Generating new ids
    cursor = connection.cursor()
    cursor.execute("truncate table data_staging.generated_ids;")
    connection.commit() 
    cursor.close()
    
    with open('./temp.csv', 'w') as file:
        for i in range(len(tessellations_frame)):
            file.write("null\n")
    copy_to_table("temp.csv", "data_staging.generated_ids(dummy_field)")

    cursor = connection.cursor()
    cursor.execute("select id from data_staging.generated_ids")
    generated_ids = [x[0] for x in cursor.fetchall()]
    connection.commit() 
    cursor.close()


    
    tile_map_id = {} # maps that keeps the correspondence between the "tile" in the dataset (here identified by building, floor, site, tile) and its newly generated ID in the database
    to_calculate_logical = {} # map that holds the ids -> [] of the logical tiles for which I will have to calculate the centroid from the associated fingerprints
    pbar = tqdm(total=len(tessellations_frame))
    
    
    file_place = open('./temp_place.csv', 'w')
    file_place_data_source = open('./temp_place_data_source.csv', 'w')
    file_contains = open('./temp_contains.csv', 'w')
    file_tile = open('./temp_tile.csv', 'w')
    
    for idx, tileinfo in enumerate(tessellations_frame.iterrows()):

        is_null = tileinfo[1].isna()

        bld = tileinfo[1][0]
        flr = tileinfo[1][1]
        site = tileinfo[1][2]
        tile = tileinfo[1][3]
        t_type = tileinfo[1][4]

        x_a = tileinfo[1][5]
        y_a = tileinfo[1][6]

        x_b = tileinfo[1][7]
        y_b = tileinfo[1][8]

        x_c = tileinfo[1][9]
        y_c = tileinfo[1][10]

        x_d = tileinfo[1][11]
        y_d = tileinfo[1][12]

        tile_map_id[str(bld) + "_" + str(flr) + "_" + str(site) + "_" + str(tile)] = generated_ids[idx]


        # Insert into place
        file_place.write(str(generated_ids[idx]) + "," + str(tile) + "\n")


        # Insert into place_data_source
        file_place_data_source.write(str(dsid) + "," + str(generated_ids[idx]) + "\n")


        # Insert into contains
        if is_null[2]: # if site is null
            # if the tile is contained in a floor
            file_contains.write(str(bldflr_map_id[str(bld) + "_" + str(flr)]) + "," + str(generated_ids[idx]) + "\n")
        else:
            # else, if the tile is contained in a site
            file_contains.write(str(bldflrsite_map_id[str(bld) + "_" + str(flr) + "_" + str(site)]) + "," + str(generated_ids[idx]) + "\n")

            
        # Insert into tile
        if t_type == 'logical':
            if is_null[5] and is_null[6]:
                # if I don't have the coordinates, then I will have to derive them from the average of the associated fingerprints
                to_calculate_logical[generated_ids[idx]] = []
                file_tile.write(str(generated_ids[idx]) + ",null,null,null,null,null,null,null,null,logical," + str(bldflrtype_map_tessid[str(bld) + "_" + str(flr) + "_logical"]) + "\n")
            else:
                file_tile.write(str(generated_ids[idx]) + "," + str(x_a) + "," + str(y_a) + ",null,null,null,null,null,null,logical," + str(bldflrtype_map_tessid[str(bld) + "_" + str(flr) + "_logical"]) + "\n")

        elif t_type in ('zone', 'grid'):
            file_tile.write(str(generated_ids[idx]) + "," + str(x_a) + "," + str(y_a) + "," + str(x_b) + "," + str(y_b) + "," + str(x_c) + "," + str(y_c) + "," + str(x_d) + "," + str(y_d) + "," + str(t_type) + "," + str(bldflrtype_map_tessid[str(bld) + "_" + str(flr) + "_" + str(t_type)]) + "\n")
        
        else:
            # crowd
            file_tile.write(str(generated_ids[idx]) + ",null,null,null,null,null,null,null,null,crowd," + str(bldflrtype_map_tessid[str(bld) + "_" + str(flr) + "_crowd"]) + "\n")



        file_imported.write(str(generated_ids[idx]) + "," + str(import_timestamp) + "," + str(dataset_name) + "," + "tile\n")

        pbar.update(1)
        
        
        
    pbar.close()   

        
    file_place.close()
    file_place_data_source.close()
    file_contains.close()
    file_tile.close()       

    copy_to_table("temp_place.csv", "data_staging.place(id, name)")   
    copy_to_table("temp_place_data_source.csv", "data_staging.place_data_source(data_source_id, place_id)") 
    copy_to_table("temp_contains.csv", "data_staging.contains(container_place_id, contained_place_id)") 
    copy_to_table("temp_tile.csv", "data_staging.tile(place_id, coordinate_a_x, coordinate_a_y, coordinate_b_x, coordinate_b_y, coordinate_c_x, coordinate_c_y, coordinate_d_x, coordinate_d_y, type, tessellation_id)") 

    
    for insid in generated_ids:
        file_imported.write(str(insid) + "," + str(import_timestamp) + "," + str(dataset_name) + "," + "tile\n")


100%|██████████| 3/3 [00:00<00:00, 540.94it/s]


In [25]:
# adjacent_to_tile

if adjacences_exists: 
    
    with open('./temp_adjacences.csv', 'w') as file:
        pbar = tqdm(total=len(adjacences_frame))
        for ind, row in adjacences_frame.iterrows():
            tile_1_id = tile_map_id[str(row.tile_1_building) + "_" + str(row.tile_1_floor) + "_" + str(row.tile_1_site) + "_" + str(row.tile_1_tile)]
            tile_2_id = tile_map_id[str(row.tile_2_building) + "_" + str(row.tile_2_floor) + "_" + str(row.tile_2_site) + "_" + str(row.tile_2_tile)]
            walkable_val = 'null' if pd.isna(row.walkable) else str(bool(int(row.walkable)))
            file.write(str(tile_1_id) + "," + str(tile_2_id) + "," + walkable_val + "," + format_nan_string_csv(row.cost) + "\n")
            pbar.update(1)
        pbar.close()   
            
    copy_to_table("temp_adjacences.csv", "data_staging.adjacent_to_tile(tile_1_place_id, tile_2_place_id, walkable, cost)")  

In [26]:
# Fingerprints
# Here I should also update the coordinates for the logical tiles that have their id in to_calculate_logical, considering only fingerprints that are both training and radio-map

# Generating new ids
cursor = connection.cursor()
cursor.execute("truncate table data_staging.generated_ids;")
connection.commit() 
cursor.close()

with open('./temp.csv', 'w') as file:
    for i in range(len(fingerprints_frame)):
        file.write("null\n")
copy_to_table("temp.csv", "data_staging.generated_ids(dummy_field)")

cursor = connection.cursor()
cursor.execute("select id from data_staging.generated_ids")
generated_ids = [x[0] for x in cursor.fetchall()]
connection.commit() 
cursor.close()




import datetime


file_fingerprint = open('./temp_fingerprint.csv', 'w')
file_gtinfo = open('./temp_gtinfo.csv', 'w')

fingerprint_code_map_id = {}
fingerprint_map_preceding_code = {}
fingerprint_map_following_code = {}
pbar = tqdm(total=len(fingerprints_frame))
for idx, fing in enumerate(fingerprints_frame.iterrows()):
    is_null = fing[1].isna()
    
    fingerprint_code_map_id[str(fing[1].fingerprint_id)] = generated_ids[idx]
    
    if not(is_null.preceded_by):
        fingerprint_map_preceding_code[generated_ids[idx]] = str(fing[1].preceded_by)
    if not(is_null.followed_by):
        fingerprint_map_following_code[generated_ids[idx]] = str(fing[1].followed_by)
        
        
    file_fingerprint.write(str(generated_ids[idx]) + "," \
                             + str(fing[1].fingerprint_id) + "," \
                             + str(dsid) + "," \
                             + ('null' if is_null.epoch else str(datetime.datetime.fromtimestamp(int(fing[1].epoch)).strftime('%Y-%m-%d %H:%M:%S')))  + "," \
                             + ('null' if is_null.coord_x or str(fing[1].is_radio_map) == "False" else str(fing[1].coord_x))  + "," \
                             + ('null' if is_null.coord_y or str(fing[1].is_radio_map) == "False" else str(fing[1].coord_y))  + "," \
                             + ('null' if is_null.coord_z or str(fing[1].is_radio_map) == "False" else str(fing[1].coord_z))  + "," \
                             +  "null," \
                             +  "null," \
                             + ('null' if is_null.user_id else str(user_code_map_id[str(fing[1].user_id)]))  + "," \
                             + ('null' if is_null.device_id else str(device_code_map_id[str(fing[1].device_id)]))  + "," \
                             + str(fing[1].is_radio_map) + "," \
                             + ('null' if str(fing[1].is_radio_map) == "False" else str(tile_map_id[str(fing[1].building) + "_" + str(fing[1].floor) + "_" + str(fing[1].site) + "_" + str(fing[1].tile)])) + "," \
                             + ('null' if is_null.set else str(fing[1].set)) + "," \
                             + ('null' if is_null.notes else str(fing[1].notes)) + "\n")
    
    file_gtinfo.write(str(generated_ids[idx]) + "," \
                         + ('null' if is_null.coord_x else str(fing[1].coord_x))  + "," \
                         + ('null' if is_null.coord_y else str(fing[1].coord_y))  + "," \
                         + ('null' if is_null.coord_z else str(fing[1].coord_z))  + "," \
                         + ('null' if is_null.tile  else str(tile_map_id[str(fing[1].building) + "_" + str(fing[1].floor) + "_" + str(fing[1].site) + "_" + str(fing[1].tile)]))  + "," \
                         + ('null' if is_null.site  else str(bldflrsite_map_id[str(fing[1].building) + "_" + str(fing[1].floor) + "_" + str(fing[1].site)]))  + "," \
                         + ('null' if is_null.floor  else str(bldflr_map_id[str(fing[1].building) + "_" + str(fing[1].floor)]))  + "," \
                         + ('null' if is_null.building  else str(building_map_id[str(fing[1].building)])) + "\n")
  

    if str(fing[1].set) == 'training' and str(fing[1].is_radio_map) == 'True':
        if tile_map_id[str(fing[1].building) + "_" + str(fing[1].floor) + "_" + str(fing[1].site) + "_" + str(fing[1].tile)] in to_calculate_logical:
            to_calculate_logical[tile_map_id[str(fing[1].building) + "_" + str(fing[1].floor) + "_" + str(fing[1].site) + "_" + str(fing[1].tile)]].append([fing[1].coord_x, fing[1].coord_y, fing[1].coord_z])

    pbar.update(1)
pbar.close()                   
       
    
file_fingerprint.close()
file_gtinfo.close()


                           
copy_to_table("temp_fingerprint.csv", "data_staging.fingerprint(id, code, data_source_id, timestamp, coordinate_x, coordinate_y, coordinate_z, preceded_by_fingerprint_id, followed_by_fingerprint_id, user_id, device_id, is_radio_map, acquired_at_tile_place_id, ml_purpose, notes)")   
copy_to_table("temp_gtinfo.csv", "data_staging.ground_truth_info(fingerprint_id, coordinate_x, coordinate_y, coordinate_z, tile_place_id, site_place_id, floor_place_id, building_place_id)") 



                           
                           
# Updating the trajectory relationship between fingerprints
for idt in fingerprint_map_preceding_code.keys():
    string_query = "UPDATE data_staging.fingerprint SET \
                        preceded_by_fingerprint_id = " + str(fingerprint_code_map_id[fingerprint_map_preceding_code[idt]]) + "\
                    WHERE id = " + str(idt) + ";"
    cursor = connection.cursor()
    cursor.execute(string_query)
    connection.commit() 
    cursor.close()   
    
for idt in fingerprint_map_following_code.keys():
    string_query = "UPDATE data_staging.fingerprint SET \
                        followed_by_fingerprint_id = " + str(fingerprint_code_map_id[fingerprint_map_following_code[idt]]) + "\
                    WHERE id = " + str(idt) + ";"
    cursor = connection.cursor()
    cursor.execute(string_query)
    connection.commit() 
    cursor.close()   

    

# Updating the logical tiles coordinates
for idt in to_calculate_logical:
    string_query = "WITH avg_coords AS ( \
                        SELECT avg(fingerprint.coordinate_x) AS avg_x, avg(fingerprint.coordinate_y) as avg_y \
                        FROM data_staging.fingerprint \
                        WHERE acquired_at_tile_place_id = " + str(idt) + " )  \
                    UPDATE data_staging.tile SET  \
                        coordinate_a_x =  avg_coords.avg_x,  \
                        coordinate_a_y =  avg_coords.avg_y \
                    FROM avg_coords \
                    WHERE tile.place_id = " + str(idt) + ";"
    cursor = connection.cursor()
    cursor.execute(string_query)
    connection.commit() 
    cursor.close()   



for insid in generated_ids:
    file_imported.write(str(insid) + "," + str(import_timestamp) + "," + str(dataset_name) + "," + "fingerprint\n")
                           
                           

100%|██████████| 9291/9291 [00:12<00:00, 759.67it/s]


In [27]:
# Access Points and Wi-Fi observations

if wifiobs_exists:

    # Generating new ids 
    cursor = connection.cursor()
    cursor.execute("truncate table data_staging.generated_ids;")
    connection.commit() 
    cursor.close()
    
    with open('./temp.csv', 'w') as file:
        for i in range(len([x for x in wifi_obs_frame.columns if 'AP-' in x])):
            file.write("null\n")
    copy_to_table("temp.csv", "data_staging.generated_ids(dummy_field)")

    cursor = connection.cursor()
    cursor.execute("select id from data_staging.generated_ids")
    generated_ids = [x[0] for x in cursor.fetchall()]
    connection.commit() 
    cursor.close()


    
    
    file_ap = open('./temp_ap.csv', 'w')
    file_apds = open('./temp_apds.csv', 'w')
    file_obs = open('./temp_obs.csv', 'w')
    file_detect = open('./temp_detect.csv', 'w')    
    
    
    # ap and ap_data_source
    ap_map_id = {} # maps that keeps the correspondence between the ap name in the dataset and its newly generated ID in the database
    for idx, apname in enumerate([x for x in wifi_obs_frame.columns if 'AP-' in x]):
        macvalue = 'null' if str(apname.split('-')[2]) == 'NULL' else str(apname.split('-')[2]) 
        ap_map_id[apname] = generated_ids[idx]
        file_ap.write(str(generated_ids[idx]) + "," + str(apname.split('-')[1]) + "," + macvalue + "\n")
        file_apds.write(str(generated_ids[idx]) + "," + str(dsid) + "\n")

    
    # Wi-Fi observation and AP detection

    pbar = tqdm(total=len(wifi_obs_frame)*len([x for x in wifi_obs_frame.columns if 'AP-' in x]))
    for _, fing in wifi_obs_frame.iterrows():
        file_obs.write(str(fingerprint_code_map_id[str(fing.fingerprint_id)]) + "\n")
        for apname in [x for x in wifi_obs_frame.columns if 'AP-' in x]:
            if not pd.isna(fing[apname]):
                file_detect.write(str(ap_map_id[apname]) + "," + str(fingerprint_code_map_id[str(fing.fingerprint_id)]) + "," + str(fing[apname]) + "\n")     
            pbar.update(1)
    pbar.close()
    
        
    
    
file_ap.close()
file_apds.close()
file_obs.close()
file_detect.close()


copy_to_table("temp_ap.csv", "data_staging.ap(id, code, mac)")   
copy_to_table("temp_apds.csv", "data_staging.ap_data_source(ap_id, data_source_id)")   
copy_to_table("temp_obs.csv", "data_staging.observation_wifi(fingerprint_id)")   
copy_to_table("temp_detect.csv", "data_staging.ap_detection(ap_id, observation_wifi_fingerprint_id, rss)")   

    
for insid in generated_ids:
    file_imported.write(str(insid) + "," + str(import_timestamp) + "," + str(dataset_name) + "," + "ap\n")
    

100%|██████████| 7442091/7442091 [02:03<00:00, 60109.03it/s] 


In [28]:
# Bluetooth devices and their observations

if bluetoothobs_exists:

    # Generating new ids 
    cursor = connection.cursor()
    cursor.execute("truncate table data_staging.generated_ids;")
    connection.commit() 
    cursor.close()
    
    with open('./temp.csv', 'w') as file:
        for i in range(len([x for x in bluetooth_obs_frame.columns if 'BL-' in x])):
            file.write("null\n")
    copy_to_table("temp.csv", "data_staging.generated_ids(dummy_field)")

    cursor = connection.cursor()
    cursor.execute("select id from data_staging.generated_ids")
    generated_ids = [x[0] for x in cursor.fetchall()]
    connection.commit() 
    cursor.close()


    
    
    file_bt = open('./temp_bt.csv', 'w')
    file_btds = open('./temp_btds.csv', 'w')
    file_obs = open('./temp_obs.csv', 'w')
    file_detect = open('./temp_detect.csv', 'w')    
    
    
    # bt and bt_data_source
    bt_map_id = {} # maps that keeps the correspondence between the bt name in the dataset and its newly generated ID in the database
    for idx, btname in enumerate([x for x in bluetooth_obs_frame.columns if 'BL-' in x]):
        macvalue = '' if str(btname.split('-')[2]) == 'NULL' else "_" + str(btname.split('-')[2]) 
        bt_map_id[btname] = generated_ids[idx]
        file_bt.write(str(generated_ids[idx]) + "," + str(btname.split('-')[1]) + macvalue + "\n")
        file_btds.write(str(generated_ids[idx]) + "," + str(dsid) + "\n")

    
    # Bluetooth observation and its detection

    pbar = tqdm(total=len(bluetooth_obs_frame)*len([x for x in bluetooth_obs_frame.columns if 'BL-' in x]))
    for _, fing in bluetooth_obs_frame.iterrows():
        file_obs.write(str(fingerprint_code_map_id[str(fing.fingerprint_id)]) + "\n")
        for btname in [x for x in bluetooth_obs_frame.columns if 'BL-' in x]:
            if not pd.isna(fing[btname]):
                file_detect.write(str(bt_map_id[btname]) + "," + str(fingerprint_code_map_id[str(fing.fingerprint_id)]) + "," + str(fing[btname]) + "\n")     
            pbar.update(1)
    pbar.close()
    
        
    
    
    file_bt.close()
    file_btds.close()
    file_obs.close()
    file_detect.close()


    copy_to_table("temp_bt.csv", "data_staging.bluetooth_device(id, name)")   
    copy_to_table("temp_btds.csv", "data_staging.bluetooth_device_data_source(bluetooth_device_id, data_source_id)")   
    copy_to_table("temp_obs.csv", "data_staging.observation_bluetooth(fingerprint_id)")   
    copy_to_table("temp_detect.csv", "data_staging.bluetooth_detection(bluetooth_device_id, observation_bluetooth_fingerprint_id, rss)")   


    for insid in generated_ids:
        file_imported.write(str(insid) + "," + str(import_timestamp) + "," + str(dataset_name) + "," + "bluetooth_device\n")
    

In [29]:
# GNSS and their observations

if gnssobs_exists:
    
    file_gnss = open('./temp_gnss.csv', 'w')
    
    # Bluetooth observation and its detection

    pbar = tqdm(total=len(gnss_obs_frame))
    for i, row in gnss_obs_frame.iterrows():
        file_gnss.write(str(fingerprint_code_map_id[str(row.fingerprint_id)]) + ",null," + format_nan_string_csv(row.latitude) + "," + format_nan_string_csv(row.longitude) + "," + format_nan_string_csv(row.elevation) + "," + format_nan_string_csv(row.num_satellites) + "\n")     
        pbar.update(1)
    pbar.close()
    
    
    file_gnss.close()

    copy_to_table("temp_gnss.csv", "data_staging.observation_gnss(fingerprint_id, is_valid, latitude, longitude, elevation, num_satellites)")   


In [30]:
# IMU and their observations

if imuobs_exists:
    
    file_imu = open('./temp_imu.csv', 'w')
    file_imu_gyro = open('./temp_imu_gyro.csv', 'w')
    file_imu_acce = open('./temp_imu_acce.csv', 'w')
    file_imu_magn = open('./temp_imu_magn.csv', 'w')
    
    # IMU observation and its detection

    unique_fings = imu_obs_frame['fingerprint_id'].drop_duplicates().values
    pbar = tqdm(total=len(unique_fings))
    for fing in unique_fings:
        fing_frame = imu_obs_frame[imu_obs_frame['fingerprint_id'] == fing]
        file_imu.write(str(fingerprint_code_map_id[str(fing)]) + ",null\n") 
        for i, row in fing_frame.iterrows():
            ep_val = str(row.epoch) if not pd.isna(row.epoch) else str(i)
            if not pd.isna(row.gyr_x):
                file_imu_gyro.write(str(fingerprint_code_map_id[str(fing)]) + "," + ep_val + "," + str(row.gyr_x) + "," + str(row.gyr_y) + "," + str(row.gyr_z) + "\n") 
            if not pd.isna(row.acc_x):
                file_imu_acce.write(str(fingerprint_code_map_id[str(fing)]) + "," + ep_val + "," + str(row.acc_x) + "," + str(row.acc_y) + "," + str(row.acc_z) + "\n") 
            if not pd.isna(row.mag_x):
                file_imu_magn.write(str(fingerprint_code_map_id[str(fing)]) + "," + ep_val + "," + str(row.mag_x) + "," + str(row.mag_y) + "," + str(row.mag_z) + "\n") 
        pbar.update(1)
    pbar.close()
 
    
    file_imu.close()
    file_imu_gyro.close()
    file_imu_acce.close()
    file_imu_magn.close()

    copy_to_table("temp_imu.csv", "data_staging.observation_imu(fingerprint_id, is_valid)")   
    copy_to_table("temp_imu_gyro.csv", "data_staging.observation_imu_gyroscope(fingerprint_id, epoch, axis_x, axis_y, axis_z)")   
    copy_to_table("temp_imu_acce.csv", "data_staging.observation_imu_accelerometer(fingerprint_id, epoch, axis_x, axis_y, axis_z)")   
    copy_to_table("temp_imu_magn.csv", "data_staging.observation_imu_magnetometer(fingerprint_id, epoch, axis_x, axis_y, axis_z)")   



In [31]:
# Verify consistency of the new data

cursor = connection.cursor()
cursor.execute("select * from check_db_consistency('data_staging')")
record = cursor.fetchall()
connection.commit() 
cursor.close()
display(record)

[]

In [32]:
# Copy the content of the data staging area into the main database schema (public)

cursor = connection.cursor()
cursor.execute("select data_staging.copy_tables()")
record = cursor.fetchall()
connection.commit() 
cursor.close()


# Write down the imported IDs
file_imported.close()
copy_to_table("temp_imported.csv", "communication.imported_ids(id, import_time, source_name, entity_name)")



# Clean the data staging area at the end of the import process
cursor = connection.cursor()
cursor.execute("select data_staging.truncate_tables('data_staging')")
record = cursor.fetchall()
connection.commit() 
cursor.close()


In [33]:
# Close the connections

connection.close()


ftp_client.close()
ssh.close()